In [2]:
import pandas as pd
from transformers import MBartTokenizer, MBartForConditionalGeneration
import random
import sys


In [14]:
rosstat = pd.read_excel('rosstat_final.xlsx')

In [18]:
rosstat.head()

,title,full_text_norm,full_text_without_tables_corrected,new_sum
0,О просроченной задолженности по заработной пла...,О просроченной задолженности по заработной пла...,О просроченной задолженности по заработной пла...,Новость сообщает о состоянии просроченной задо...
1,О потребительских ценах на нефтепродукты с 25 ...,О потребительских ценах на нефтепродукты с 25 ...,О потребительских ценах на нефтепродукты с 25 ...,Новость сообщает о динамике потребительских це...
2,О промышленном производстве в январе-сентябре ...,О промышленном производстве в январе-сентябре ...,NaN,NaN
3,О просроченной задолженности по заработной пла...,О просроченной задолженности по заработной пла...,О просроченной задолженности по заработной пла...,Новость сообщает о состоянии просроченной задо...
4,О потребительских ценах на нефтепродукты с 8 п...,О потребительских ценах на нефтепродукты с 8 п...,О потребительских ценах на нефтепродукты с 8 п...,Новость сообщает о динамике потребительских це...


In [ ]:
# !pip install transformers sentencepiece datasets torch rouge-score nltk auto_gptq optimum
# !pip install auto_gptq

import pandas as pd
import torch
from transformers import AutoTokenizer, BartForConditionalGeneration, AutoModelForSeq2SeqLM  # Изменено
from auto_gptq import AutoGPTQForCausalLM
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

df = pd.read_excel('rosstat_final.xlsx')
df = df[df['new_sum'].notna()]
df = df.dropna(subset=['full_text_without_tables_corrected', 'new_sum'])

# Обновленная конфигурация моделей
models = {
    "ruBart": {
        "model_name": "IlyaGusev/mbart_ru_sum_gazeta",
        "type": "seq2seq",
        "class": "bart"  # Флаг для специальной загрузки
    },
    "ruT5": {
        "model_name": "IlyaGusev/rut5_base_sum_gazeta",
        "type": "seq2seq"
    },
    "FRED-T5": {
        "model_name": "ai-forever/FRED-T5-large",
        "type": "seq2seq"
    },
    "Qwen-Q3": {
        "model_name": "Qwen/Qwen1.5-0.5B-Chat-GPTQ",  # Облегченная версия
        "type": "causal",
        "quant": True
    }
}

def generate_seq2seq(model, tokenizer, text, max_length=512):
    inputs = tokenizer(
        text,
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
        padding="max_length"
    ).to(model.device)

    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=150,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def generate_qwen(model, tokenizer, text, max_length=512):
    prompt = f"""
    <|im_start|>system
    Вы - AI ассистент для суммаризации новостей. Создайте краткое содержание следующего новостного текста на русском языке.
    <|im_end|>
    <|im_start|>user
    {text}
    <|im_end|>
    <|im_start|>assistant
    """

    inputs = tokenizer(
        prompt,
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
        return_attention_mask=True
    ).to(model.device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=150,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False
    )
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.split('<|im_end|>', 1)[0].strip()

# Инициализация метрик
rouge = load_metric("rouge")
smooth = SmoothingFunction().method4
results = []

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import pandas as pd
import torch
from transformers import MBartTokenizer, MBartForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from auto_gptq import AutoGPTQForCausalLM
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

df = pd.read_excel('rosstat_final.xlsx')
df = df[df['new_sum'].notna()]
df = df.dropna(subset=['full_text_without_tables_corrected', 'new_sum'])


# Конфигурация моделей
models = {
    "mbart-large": {
        "model_name": "facebook/mbart-large-50",
        "type": "seq2seq"
    },
    "ruT5": {
        "model_name": "IlyaGusev/rut5_base_sum_gazeta",
        "type": "seq2seq"
    },
    "FRED-T5": {
        "model_name": "ai-forever/FRED-T5-large",
        "type": "seq2seq"
    },
    "rugpt3": {
        "model_name": "sberbank-ai/rugpt3large_based_on_gpt2",
        "type": "causal"
    }
}

# Функции генерации
def generate_seq2seq(model, tokenizer, text, max_length=512):
    inputs = tokenizer(
        text,
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
        padding="max_length"
    ).to(model.device)

    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=150,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def generate_mbart(text):
    tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-50")
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    input_ids = tokenizer(
        [text],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=150,
        no_repeat_ngram_size=4
    )[0]
    return tokenizer.decode(output_ids, skip_special_tokens=True)

def generate_fredt5(model, tokenizer, text, max_length=512):
    prompt = f"Суммаризируй текст: {text}"
    inputs = tokenizer(
        prompt,
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
        padding="max_length"
    ).to(model.device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=150,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_rugpt3(model, tokenizer, text, max_length=1024):
    prompt = f"Кратко суммаризируй следующую новость: {text}. Не забудь в начале сказать, о чем представленная новость"

    inputs = tokenizer(
        prompt,
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
        return_attention_mask=True
    ).to(model.device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=150,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False
    )
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

# Инициализация метрик
rouge = load_metric("rouge")
smooth = SmoothingFunction().method4
results = []

# Обработка моделей
for model_name, config in models.items():
    print(f"\n=== Обработка модели: {model_name} ===")

    try:
        if model_name == "mbart-large":
            tokenizer = MBartTokenizer.from_pretrained(config["model_name"])
            model = MBartForConditionalGeneration.from_pretrained(config["model_name"])
        elif model_name in ["ruT5", "FRED-T5"]:
            tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
            model = AutoModelForSeq2SeqLM.from_pretrained(config["model_name"])
        elif model_name == "rugpt3":
            tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
            model = AutoModelForCausalLM.from_pretrained(config["model_name"], device_map="auto")

        if model_name != "rugpt3":
            device = "cuda" if torch.cuda.is_available() else "cpu"
            model.to(device)

        predictions, references = [], []
        meteor_scores, bleu_scores = [], []

        for _, row in df.iterrows():
            text = row['full_text_without_tables_corrected']
            ref_summary = row['new_sum']

            if model_name == "mbart-large":
                gen_summary = generate_mbart(text)
            elif model_name == "FRED-T5":
                gen_summary = generate_fredt5(model, tokenizer, text)
            elif model_name == "ruT5":
                gen_summary = generate_seq2seq(model, tokenizer, text)
            elif model_name == "rugpt3":
                gen_summary = generate_rugpt3(model, tokenizer, text)

            predictions.append(gen_summary)
            references.append(ref_summary)

            ref_tokens = [word_tokenize(ref_summary.lower())]
            gen_tokens = word_tokenize(gen_summary.lower())
            bleu_scores.append(sentence_bleu(ref_tokens, gen_tokens, smoothing_function=smooth))
            meteor_scores.append(meteor_score(ref_tokens, gen_tokens))

        rouge_scores = rouge.compute(
            predictions=predictions,
            references=references,
            rouge_types=["rouge1", "rouge2", "rougeL"]
        )

        results.append({
            "model": model_name,
            "rouge1": rouge_scores["rouge1"].mid.fmeasure,
            "rouge2": rouge_scores["rouge2"].mid.fmeasure,
            "rougeL": rouge_scores["rougeL"].mid.fmeasure,
            "bleu": sum(bleu_scores) / len(bleu_scores),
            "meteor": sum(meteor_scores) / len(meteor_scores)
        })

    except Exception as e:
        print(f"Ошибка в модели {model_name}: {str(e)}")
        results.append({
            "model": model_name,
            "rouge1": None,
            "rouge2": None,
            "rougeL": None,
            "bleu": None,
            "meteor": None
        })

    if 'model' in locals():
        del model
    torch.cuda.empty_cache()

results_df = pd.DataFrame(results)
print("\nИтоговые результаты:")
results_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.



=== Обработка модели: mbart-large ===


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result


=== Обработка модели: ruT5 ===

=== Обработка модели: FRED-T5 ===

=== Обработка модели: rugpt3 ===

Итоговые результаты:


,model,rouge1,rouge2,rougeL,bleu,meteor
0,mbart-large,0.260625,0.123635,0.254954,0.004893,0.087498
1,ruT5,0.457279,0.282724,0.429501,0.055240,0.187099
2,FRED-T5,0.439689,0.267484,0.401804,0.088597,0.226078
3,rugpt3,0.291426,0.156545,0.255202,0.057392,0.162995


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset
import pandas as pd
import numpy as np

df = pd.read_excel('rosstat_final.xlsx')
df = df[df['new_sum'].notna()]
df = df.dropna(subset=['full_text_without_tables_corrected', 'new_sum'])


train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = ["summarize: " + text for text in examples["full_text_without_tables_corrected"]]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["new_sum"],
            max_length=150,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

training_args = Seq2SeqTrainingArguments(
    output_dir="./rut5_finetuned",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_steps=100,
    save_steps=500,
    warmup_steps=500,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model("./rut5_finetuned_final")


Map:   0%|          | 0/228 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/26 [00:00<?, ? examples/s]

/tmp/ipython-input-6-3719133225.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,nan
2,0.000000,nan
3,0.000000,nan


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
def evaluate_model(model, tokenizer, dataset, original_df):
    predictions = []
    references = []
    meteor_scores = []
    bleu_scores = []
    smooth = SmoothingFunction().method4

    for i, example in enumerate(dataset):
        input_text = "summarize: " + example["full_text_without_tables_corrected"]
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=4,
            early_stopping=True
        )

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        ref = example["new_sum"]

        predictions.append(pred)
        references.append(ref)

        ref_tokens = [word_tokenize(ref.lower())]
        pred_tokens = word_tokenize(pred.lower())
        bleu_scores.append(sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth))
        meteor_scores.append(meteor_score(ref_tokens, pred_tokens))

    rouge = load_metric("rouge")
    rouge_scores = rouge.compute(predictions=predictions, references=references, rouge_types=["rouge1", "rouge2", "rougeL"])

    results_df = pd.DataFrame({
        "text": [x["full_text_without_tables_corrected"] for x in dataset],
        "reference": references,
        "prediction": predictions
    })

    avg_metrics = {
        "rouge1": rouge_scores["rouge1"].mid.fmeasure,
        "rouge2": rouge_scores["rouge2"].mid.fmeasure,
        "rougeL": rouge_scores["rougeL"].mid.fmeasure,
        "bleu": np.mean(bleu_scores),
        "meteor": np.mean(meteor_scores)
    }

    return results_df, avg_metrics

# Загрузка дообученной модели
finetuned_model = T5ForConditionalGeneration.from_pretrained("./rut5_finetuned_final").to("cuda" if torch.cuda.is_available() else "cpu")

test_results, test_metrics = evaluate_model(finetuned_model, tokenizer, test_dataset, test_df)

print("\nМетрики на тестовых данных после дообучения:")
print(f"ROUGE-1: {test_metrics['rouge1']:.4f}")
print(f"ROUGE-2: {test_metrics['rouge2']:.4f}")
print(f"ROUGE-L: {test_metrics['rougeL']:.4f}")
print(f"BLEU: {test_metrics['bleu']:.4f}")
print(f"METEOR: {test_metrics['meteor']:.4f}")

print("\nПримеры предсказаний:")
test_results


Метрики на тестовых данных после дообучения:
ROUGE-1: 0.4031
ROUGE-2: 0.2158
ROUGE-L: 0.3853
BLEU: 0.0451
METEOR: 0.1765

Примеры предсказаний:


,text,reference,prediction
0,О рынке нефти в 2022 году\n\nО рынке нефти\nв ...,Новость посвящена обзору рынка нефти за 2022 г...,О рынке нефти в 2022 году О рынке нефти в 2022...
1,Об использовании валового внутреннего продукта...,Новости об использовании валового внутреннего ...,Объем ВВП России за II квартал 2024 года соста...
2,Об объеме производства нефтепродуктов с 29 янв...,Новость сообщает об изменениях в производстве ...,С 29 января по 4 февраля 2024 года рост цен на...
3,О рынке нефти в январе-ноябре 2022 года\n\nО р...,Новость о состоянии рынка нефти за январь-нояб...,О рынке нефти в январе-ноябре 2022 года Добыча...
4,О потребительских ценах на нефтепродукты с 18 ...,Новость сообщает о динамике потребительских це...,Динамика потребительских цен на бензин автомоб...
5,О динамике цен на бензин автомобильный и ресур...,Новость посвящена изменению цен на автомобильн...,Потребительские цены на бензин автомобильный в...
6,О промышленном производстве в 2022 году\nИндек...,Новость об индексе промышленного производства ...,О промышленном производстве в 2022 году Индекс...
7,О динамике цен на бензин автомобильный в апрел...,Новость посвящена изменению цен на автомобильн...,Потребительские цены на бензин автомобильный в...
8,О динамике цен на бензин автомобильный в авгус...,Новость посвящена изменению цен на автомобильн...,В августе 2024 года потребительские цены на бе...
9,О просроченной задолженности по заработной пла...,Новости об уровне просроченной задолженности п...,Суммарная задолженность по заработной плате на...


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk import word_tokenize
import numpy as np
import pandas as pd
from tqdm import tqdm


model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def evaluate_base_model(model, tokenizer, test_df):
    predictions = []
    references = test_df['target_text'].tolist()
    meteor_scores = []
    bleu_scores = []
    smooth = SmoothingFunction().method4

    for text in tqdm(test_df['input_text'], desc="Оценка модели"):
        input_text = "summarize: " + text
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)

        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=4,
            early_stopping=True
        )

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred)

    for ref, pred in zip(references, predictions):
        ref_tokens = [word_tokenize(str(ref).lower())]
        pred_tokens = word_tokenize(str(pred).lower())

        bleu_scores.append(sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth))
        meteor_scores.append(meteor_score(ref_tokens, pred_tokens))

    rouge = load_metric("rouge")
    rouge_scores = rouge.compute(
        predictions=predictions,
        references=references,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )

    results_df = pd.DataFrame({
        "input_text": test_df['input_text'],
        "target_text": references,
        "prediction": predictions
    })

    avg_metrics = {
        "rouge1": rouge_scores["rouge1"].mid.fmeasure,
        "rouge2": rouge_scores["rouge2"].mid.fmeasure,
        "rougeL": rouge_scores["rougeL"].mid.fmeasure,
        "bleu": np.mean(bleu_scores),
        "meteor": np.mean(meteor_scores)
    }

    return results_df, avg_metrics

base_results, base_metrics = evaluate_base_model(model, tokenizer, test_df)

print("\nМетрики базовой модели ruT5 на тестовых данных:")
print(f"ROUGE-1: {base_metrics['rouge1']:.4f}")
print(f"ROUGE-2: {base_metrics['rouge2']:.4f}")
print(f"ROUGE-L: {base_metrics['rougeL']:.4f}")
print(f"BLEU: {base_metrics['bleu']:.4f}")
print(f"METEOR: {base_metrics['meteor']:.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Оценка модели: 100%|██████████| 26/26 [01:03<00:00,  2.43s/it]



Метрики базовой модели ruT5 на тестовых данных:
ROUGE-1: 0.4021
ROUGE-2: 0.2124
ROUGE-L: 0.3779
BLEU: 0.0406
METEOR: 0.1671
